In [193]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

In [194]:
# source: https://dune.com/queries/317386/604040
prepend = 'csv_exports/nft_sales/'
filen = 'marketplace_stats_'
date = '23_01_23'

mktpl_df = pd.read_csv(prepend + filen + date + '.csv')
mktpl_df['dt'] = pd.to_datetime(mktpl_df['dt']).dt.tz_localize(None)

# print(mktpl_df.dtypes)


In [195]:
#Pre-Quests
min_date = pd.to_datetime('2022-10-01', format='%Y-%m-%d')
max_date = pd.to_datetime('2023-01-01', format='%Y-%m-%d')
#Post-Quests
# min_date = pd.to_datetime('2023-01-18', format='%Y-%m-%d')
# max_date = pd.to_datetime("today")
# print(min_date)

mktpl_df = mktpl_df[(mktpl_df['dt'] >= pd.to_datetime(min_date) ) \
                        & (mktpl_df['dt'] < pd.to_datetime(max_date) )\
                        ]

total_df = mktpl_df.copy()
total_df['marketplace'] = 'Total' #override for sums

#add extra "totals" row
total_df = pd.concat([mktpl_df,total_df])

In [198]:
# segment by marketplace

mktpl_agg = total_df.groupby(['marketplace']).agg({
    'num_trades':'sum'
    ,'eth_value_traded':'sum'
    ,'usd_value_traded':'sum'
    ,'dt':'nunique'
})


mktpl_agg.reset_index(inplace=True)
mktpl_agg = mktpl_agg.rename(columns={'dt':'num_days'})
# display(mktpl_agg)

for c in mktpl_agg.columns:
    if c == 'marketplace':
        continue
    else:
        mktpl_agg[c+'_avg'] = mktpl_agg[c] / mktpl_agg['num_days']
        mktpl_agg[c+'_annualized'] = ( mktpl_agg[c] / mktpl_agg['num_days'] ) * 365

        total_num = mktpl_agg.loc[mktpl_agg['marketplace'] == 'Total'][c].iloc[0]
        mktpl_agg[c+'_pct_share'] = mktpl_agg[c] / total_num

display(mktpl_agg)

avg_trade = mktpl_agg[['marketplace','usd_value_traded_avg','num_trades_avg']]
avg_trade['avg_trade_side'] = avg_trade['usd_value_traded_avg'] / avg_trade['num_trades_avg']
display(avg_trade)

,marketplace,num_trades,eth_value_traded,usd_value_traded,num_days,num_trades_avg,num_trades_annualized,num_trades_pct_share,eth_value_traded_avg,eth_value_traded_annualized,eth_value_traded_pct_share,usd_value_traded_avg,usd_value_traded_annualized,usd_value_traded_pct_share,num_days_avg,num_days_annualized,num_days_pct_share
0,OpenSea,56684,439.796908,5.901496e+05,92,616.130435,2.248876e+05,0.150499,4.780401,1744.846427,0.129884,6414.669135,2.341354e+06,0.1332,1.0,365.0,1.0
1,Quix,319956,2946.276847,3.840408e+06,92,3477.782609,1.269391e+06,0.849501,32.024748,11689.033142,0.870116,41743.566887,1.523640e+07,0.8668,1.0,365.0,1.0
2,Total,376640,3386.073754,4.430558e+06,92,4093.913043,1.494278e+06,1.000000,36.805150,13433.879569,1.000000,48158.236022,1.757776e+07,1.0000,1.0,365.0,1.0


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_85596/1383539100.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avg_trade['avg_trade_side'] = avg_trade['usd_value_traded_avg'] / avg_trade['num_trades_avg']


,marketplace,usd_value_traded_avg,num_trades_avg,avg_trade_side
0,OpenSea,6414.669135,616.130435,10.411219
1,Quix,41743.566887,3477.782609,12.002926
2,Total,48158.236022,4093.913043,11.763375
